In [1]:
from tsrrt.kinematics import PandaKinematics
from tsrrt.collision import PandaCollision
import pybullet as p
import pybullet_data
import numpy as np
from spatial_math_mini import *
from tsrrt.data_structure import *
from tsrrt.RRT import *

pybullet build time: Oct 11 2021 20:59:39


In [2]:
def view_pose(pos, ori):
    T = SE3.qtn_trans(ori, pos).T
    length = 0.1
    xaxis = np.array([length, 0, 0, 1])
    yaxis = np.array([0, length, 0, 1])
    zaxis = np.array([0, 0, length, 1])
    T_axis = np.array([xaxis, yaxis, zaxis]).T
    axes = T @ T_axis
    orig = T[:3,-1]
    xaxis = axes[:-1,0]
    yaxis = axes[:-1,1]
    zaxis = axes[:-1,2]
    x = p.addUserDebugLine(orig,xaxis, lineColorRGB=[1,0,0], lineWidth=5)
    y = p.addUserDebugLine(orig,yaxis, lineColorRGB=[0,1,0], lineWidth=5)
    z = p.addUserDebugLine(orig,zaxis, lineColorRGB=[0,0,1], lineWidth=5)
    pose_id = [x, y, z]
    return pose_id

def view_node(node):
    view_pose(node.pos, [1,0,0,0])

def clear(rng=None):
    if rng is None:
        rng = range(100)
    for i in rng:
        p.removeUserDebugItem(i)

In [3]:
gui_uid = p.connect(p.GUI)
rate = 240.
p.setTimeStep(1/rate)
p.resetSimulation() #init

#
p.setAdditionalSearchPath(pybullet_data.getDataPath()) # for loading plane
plane_id = p.loadURDF("plane.urdf") # load plane
lego = p.loadURDF("lego/lego.urdf")
p.resetBasePositionAndOrientation(lego,[0.5,0.1,0.01], [0,0,0,1])

checker = PandaCollision(p, gui_uid)

### C-space RRT

In [80]:
start = NodeJoint(checker._home_positions)
goal = NodeJoint.random()
# load rrt
rrt = RRT(start, goal, checker)

In [81]:
checker.set_joint_positions(goal.q)

In [82]:
success, plan = rrt.plan()

planning success
elapsed time : 0.320143
nodes:802


In [83]:
i=0
checker.set_joint_positions(plan[i].q)

In [153]:
i += 1
checker.set_joint_positions(plan[i].q)
print(i+1,"/", len(plan)+1)

IndexError: list index out of range

### Task-space RRT (R3)

In [154]:
checker.set_home_positions()
pos, _ = checker.get_ee_pose()
q = np.array(checker._home_positions)
start = NodeTaskPos(pos, q)
#
q_goal = NodeJoint.random().q
pos, ori = checker.FK(q_goal)
goal = NodeTaskPos(pos, q_goal)

# load rrt
rrt = RRT(start, goal, checker)

In [155]:
view_pose(goal.pos, [1,0,0,0])
checker.set_joint_positions(q_goal)

In [156]:
success, plan = rrt.plan()

planning success
elapsed time : 0.061662999999999996
nodes:125


In [157]:
i=0
checker.set_joint_positions(plan[i].q)

In [195]:
i += 1
checker.set_joint_positions(plan[i].q)
print(i+1,"/", len(plan)+1)

IndexError: list index out of range

### Task-space RRT (R3, SO3)

In [4]:
NodeTaskPosOri.trans_max = 0.1
NodeTaskPosOri.rot_max = 0.1
NodeTaskPosOri.eps = 0.01

In [5]:
checker.set_home_positions()
pos, ori = checker.get_ee_pose()
q = np.array(checker._home_positions)
start = NodeTaskPosOri(pos, ori, q)
#
q_goal = NodeJoint.random().q
pos, ori = checker.FK(q_goal)
goal = NodeTaskPosOri(pos, ori, q_goal)

# load rrt
rrt = RRT(start, goal, checker)

In [29]:
view_pose(goal.pos, goal.ori)
checker.set_joint_positions(q_goal)

In [30]:
success, plan = rrt.plan()

planning fail
elapsed time : 5.044366
nodes:2982


In [8]:
import numpy as np
from tsrrt.data_structure import NodeJoint, Tree
from spatial_math_mini import SO3, SE3
import datetime

class RRT:
    def __init__(self, start, goal, checker):
        self._tree = Tree(start)
        self._start = start
        self._goal = goal
        self._last_node = None
        self._node_class = type(start)
        self._checker = checker
        
        self._goal_bias = 0.05
        
    def extend(self, node_rand):
        node_near = self._tree.nearest(node_rand)
        if self._node_class.distance(node_rand, node_near) < self._node_class.eps:
            # same node
            return
        
        parent = node_near
        node_new, is_stop = self._node_class.control(parent, node_rand, self._checker)
        if is_stop:
            return
        while node_new != node_rand:
            self._checker.set_joint_positions(node_new.q)
            if (not node_new.is_valid(self._checker)):
                break
            self._tree.add(parent, node_new)
            
            if (self._node_class.distance(self._goal, node_new) <= self._node_class.eps):
                self._last_node = node_new
                break
            parent = node_new
            node_new, is_stop = self._node_class.control(parent, node_rand, self._checker)
            if is_stop:
                break
    
    def is_goal_found(self):
        return self._last_node is not None
    
    def plan(self, max_time=10000000000000.0):
        self._last_node = None
        start_time = datetime.datetime.now()
        while not self.is_goal_found():
            if np.random.random() > self._goal_bias:
                node_rand = self._node_class.random()
            else:
                node_rand = self._goal
            view_pose(node_rand.pos, node_rand.ori)
            self.extend(node_rand)

            time_delta = (datetime.datetime.now() - start_time)
            elapsed = time_delta.seconds + 0.000001 * time_delta.microseconds
            if elapsed > max_time:
                break

        if self.is_goal_found():
            print("planning success")
        else:
            print("planning fail")
        print("elapsed time : {}".format(elapsed))
        print("nodes:{}".format(self._tree._num))
        return self.is_goal_found(), self.path()

    def path(self):
        if not self.is_goal_found():
            return []
        path = [self._last_node]
        child_idx = self._last_node.idx
        while True:
            parent_idx = self._tree._parent_of[child_idx]
            if parent_idx is None:
                break
            path.append(self._tree._data[parent_idx])
            child_idx = parent_idx
        return path[::-1]

    def reset(self):
        self._tree = Tree(self._start)
        self._last_node = None

In [9]:
# load rrt
rrt = RRT(start, goal, checker)

In [10]:
rrt.plan()

Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1078, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 297, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "/home/kh11kim/.local/lib/python3.8/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 1976, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "/home/kh11kim/.local/lib/python3.8/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2011, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
Node

In [13]:
node, success = NodeTaskPosOri.control(start, goal, checker)

In [40]:
node, success = NodeTaskPosOri.control(node, goal, checker)
checker.set_joint_positions(node.q)

In [41]:
checker.set_joint_positions(start.q)

In [30]:
checker.set_joint_positions(node.q)

In [44]:
view_pose(start.pos, start.ori)

[9, 10, 11]

In [856]:
NodeTaskPosOri.trans_max = 0.1
NodeTaskPosOri.rot_max = 2

In [44]:
clear()

In [8]:
goal = NodeTaskPosOri(start.pos+np.array([0.2,0.2,0.2]), SO3.random().to_qtn())

In [45]:
view_pose(start.pos, start.ori)
view_pose(goal.pos, goal.ori)

[15, 16, 17]

In [80]:
node, suc = NodeTaskPosOri.control(start, goal, checker)

In [109]:
node, suc = NodeTaskPosOri.control(node, goal, checker)
checker.set_joint_positions(node.q)
print(node._m)

39.55323385227775


In [81]:
checker.set_joint_positions(node.q)

In [50]:
node.q

AttributeError: 'tuple' object has no attribute 'q'

In [13]:
pos, ori = start.pos, start.ori
q_new = start.q

In [41]:
SE3_1 = SE3(ori, pos)
SE3_2 = SE3(goal.ori, goal.pos)
screw, angle = (SE3_1.inv() @ SE3_2).to_twistangle()
twist = screw*angle
diff = NodeTaskPosOri.crop_within_limits(twist)
jac_pinv = np.linalg.pinv(checker.get_body_jacobian(q_new))
q_new = jac_pinv @ diff + q_new
pos, ori = checker.get_ee_pose()
checker.set_joint_positions(q_new)

In [43]:
pos, ori = checker.get_ee_pose()
view_pose(pos, ori)

[9, 10, 11]

In [1809]:
screw, angle

array([ 0.73483486, -0.61574523,  0.28438624, -0.05723631, -0.12342765,
        0.07559676])

In [1567]:
checker.set_joint_positions(start.q)
view_pose(start.pos, start.ori)

[15, 16, 17]

In [217]:
i=0
checker.set_joint_positions(plan[i].q)

In [259]:
i += 1
checker.set_joint_positions(plan[i].q)
print(i+1,"/", len(plan)+1)

43 / 45


In [34]:
i=0
checker.set_joint_positions(rrt._tree._data[i].q)

In [90]:
i += 1
checker.set_joint_positions(rrt._tree._data[i].q)
print(i+1,"/", len(plan)+1)

57 / 1


In [7]:
panda.set_home_positions()

In [8]:
panda.set_joint_positions(NodeJoint.random())

In [11]:
rand = NodeJoint.random()

In [12]:
np.linalg.norm(root.q - rand.q)

3.8010215297231573

In [11]:
root - rand

2.490236950544942

In [14]:
collision_checker.set_home_positions()
pos = StateSpace._get_valid_sample()
view_pose(pos, [1,0,0,0])

[3, 4, 5]

In [23]:
a = Node.random(set_q=True)

In [24]:
a.q

array([ 0.28907159,  0.30887164,  0.2805836 , -2.14880861,  0.11704414,
        1.33202861,  0.78539816])

In [12]:
a[0]

TypeError: 'NoneType' object is not subscriptable